In [1]:
from unsloth import FastLanguageModel                                                                                                                                                                                                                                           
from transformers import AutoTokenizer


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 02-17 19:04:32 [__init__.py:244] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annota

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B",
)

==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.1. vLLM: 0.9.2+computecanada.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [8]:
from tuning.utils.utils import chat_template_func

tokenizer = chat_template_func(tokenizer, "llama-3.1")
print(tokenizer.chat_template)

{% if 'role' in messages[0] %}{{- bos_token }}{%- if messages[0]['role'] == 'system' %}    {%- set system_message = messages[0]['content'] %}    {%- set messages = messages[1:] %}{%- else %}    {%- set system_message = "" %}{%- endif %}{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}{{- system_message }}{{- "<|eot_id|>" }}{%- for message in messages %}    {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] + '<|eot_id|>' }}{%- endfor %}{%- if add_generation_prompt %}    {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{%- endif %}{% else %}{{- bos_token }}{%- if messages[0]['from'] == 'system' %}    {%- set system_message = messages[0]['value'] %}    {%- set messages = messages[1:] %}{%- else %}    {%- set system_message = "" %}{%- endif %}{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}{{- system_message }}{{- "<|eot_id|>" }}{%- for message in messages %}    {{- '<|start_header_id|>' + message['from'] + '<|end_header_id|>\n\

In [5]:
from datasets import load_from_disk
from tuning.utils.utils import apply_chat_template
from tuning.config import DATASETS_DIR

/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning


## SFT Datasets

In [6]:
# Load SFT datasets
# sft_gsm8k = load_from_disk(f"{DATASETS_DIR}/sft-gsm8k")
sft_tuluif = load_from_disk(f"{DATASETS_DIR}/sft-tuluif")

# print("=== SFT GSM8K ===")
# print(sft_gsm8k)
# print(f"\nExample row:\n{sft_gsm8k['train'][0]}")

print("\n=== SFT TuluIF ===")
print(sft_tuluif)
print(f"\nExample row:\n{sft_tuluif['train'][0]}")


=== SFT TuluIF ===
DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 29780
    })
    test: Dataset({
        features: ['id', 'prompt', 'messages', 'constraints'],
        num_rows: 200
    })
})

Example row:
{'id': 'personas_IF_1s84skm7dqifreqgvsgzyxdf', 'prompt': 'Provide a detailed analysis of Candace Parker\'s defensive techniques in her recent games, excluding the words "aggressive" and "blocking", in the format of a sports commentary script.', 'messages': [{'content': 'You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions', 'role': 'system'}, {'content': 'Provide a detailed analysis of Candace Parker\'s defensive techniques in her recent games, excluding the words "aggressive" and "blocking", in the format of a sports commentary script.', 'role': 'user'}, {'content': "[Sports Commentary Script]\n\n[Opening Scene: A packed basketball arena, with fans e

In [9]:
print(tokenizer.apply_chat_template(sft_tuluif["train"][0]["messages"], tokenize = False, add_generation_prompt = False))

<|im_start|>system
You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions<|im_end|>
<|im_start|>user
Provide a detailed analysis of Candace Parker's defensive techniques in her recent games, excluding the words "aggressive" and "blocking", in the format of a sports commentary script.<|im_end|>
<|im_start|>assistant
[Sports Commentary Script]

[Opening Scene: A packed basketball arena, with fans eagerly awaiting the analysis of Candace Parker’s recent performances on the court.]

Commentator 1: Welcome back, basketball fans! Tonight, we're diving into the defensive prowess of Candace Parker, who has been a standout in her recent games. Her ability to impact the game on the defensive end is truly remarkable, and we've got a lot to unpack.

Commentator 2: That's right! Candace has been showcasing some exceptional skills, especially in how she reads the game. Her anticipation is second to none. You can see it in how she positions

In [10]:
sft_tuluif_templated = apply_chat_template(tokenizer, sft_tuluif)
print("=== Templated SFT TuluIF ===")
print(sft_tuluif_templated["train"][0]["text"])

Map:   0%|          | 0/29780 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

=== Templated SFT TuluIF ===
<|im_start|>system
You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions<|im_end|>
<|im_start|>user
Provide a detailed analysis of Candace Parker's defensive techniques in her recent games, excluding the words "aggressive" and "blocking", in the format of a sports commentary script.<|im_end|>
<|im_start|>assistant
[Sports Commentary Script]

[Opening Scene: A packed basketball arena, with fans eagerly awaiting the analysis of Candace Parker’s recent performances on the court.]

Commentator 1: Welcome back, basketball fans! Tonight, we're diving into the defensive prowess of Candace Parker, who has been a standout in her recent games. Her ability to impact the game on the defensive end is truly remarkable, and we've got a lot to unpack.

Commentator 2: That's right! Candace has been showcasing some exceptional skills, especially in how she reads the game. Her anticipation is second to none. You ca

## IFEval Generation

In [ ]:
from tuning.data.test_dataset import get_ifeval_test_dataset
from tuning.inference.vllm_utils import load_vlm_model, generate_responses_vllm

ifeval_dataset = get_ifeval_test_dataset()
print(f"IFEval test set size: {len(ifeval_dataset)}")
print(f"\nExample prompt:\n{ifeval_dataset[0]['prompt']}")

# Generate using vLLM (single example)
example = ifeval_dataset.select([0])
model_name = "llama3-1B"
llm, sampling_params = load_vlm_model(model_name, n=1, temperature=0.7)
results = generate_responses_vllm(llm, sampling_params, example["prompt"], example["messages"], chat_template="chatml")
print(f"\nGenerated response:\n{results[0]['response']}")

IFEval test set size: 541

Example prompt:
Write a 300+ word summary of the wikipedia page "https://en.wikipedia.org/wiki/Raymond_III,_Count_of_Tripoli". Do not use any commas and highlight at least 3 sections that has titles in markdown format, for example *highlighted section part 1*, *highlighted section part 2*, *highlighted section part 3*.
Loading model from /home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B
Using GPU memory utilization: 0.8
INFO 02-17 19:25:52 [config.py:841] This model supports multiple tasks: {'reward', 'embed', 'generate', 'classify'}. Defaulting to 'generate'.


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 02-17 19:25:52 [config.py:3368] Downcasting torch.float32 to torch.bfloat16.
INFO 02-17 19:25:52 [config.py:1472] Using max model len 131072
INFO 02-17 19:25:54 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 02-17 19:25:55 [__init__.py:2662] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reason: CUDA is initialized


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/python/3.11.5/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2023/x86-64-v3/Compiler/gcccore/python/3.11.5/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/vllm/v1/engine/__init__.py", line 13, in <module>
    from vllm.multimodal import MultiModalKwargs
  File "/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/vllm/multimodal/__init__.py", line 8, in <module>
    from .registry import MultiModalRegistry
  File "/home/shougan/p